# AES128 Counter Mode Decrypt Example

#### we'll use 3 texts:
##### PlainText1 (Cat.txt)
##### CipherText1 (Encrypted_Cat.txt)
##### CipherText2 (Challenge.txt)


This is a same key scenario. The key used is unknown. All we know is that AES128 in counter mode is used (without Nonce) and the two ciphertexts are encrypted with the same key. The Objective is to retrieve the Plaintext of CipherText2 
(Challenge.txt)

#### IMPORTANT: if you run it on binder it may not work (probably there's something wrong with the readfile). This notebook is intended for \"Read Only\". If you want to try the code, download the files and run on local jupyter notebook

In [6]:
#!/usr/bin/python3.7

# Install numpy package in the current Jupyter kernel if not present
import sys
try:
    import numpy
except ImportError:
    !conda install --yes --prefix {sys.prefix} numpy

import numpy as np
import collections

cat = np.fromfile('cat.txt',np.dtype('B'))
encrypted_cat = np.fromfile('encrypted_cat.txt',np.dtype('B'))
challenge = np.fromfile('challenge.txt',np.dtype('B'))

In [7]:
keystream = np.bitwise_xor(cat,encrypted_cat[:len(cat)])
print(keystream[:1000].tostring()) #output the keystream

b'\xb4\x95\xd9\x04\xc6@\xe1\x1b\x06hv\xd90\xc4\xae#v\x7f\xa9x\xed\x86\x0b\x97\xe2n\x9b\xd4\xc6\xac\xbeI\xe01x\xbfL>>G\x9c\r\xfbU5\xfe\xb0\xc3`\xdb\xc9\xcci?n\x81_\xd3\x0c\x07W}\x8cG\xad\r-h\x9cE\xdcJ?\xf3\xcc\x0b0\x16\nt\xc1\x01g\x8c\x8a%\xf0\xea\x88\xc5Gw\xd5\xd8\xa0\xe2A\xd81\xe0}\xbd\xc3zz\xe2\xe6~\x001p\n\xf8\xe0NE\x88,\xdcXi\xc6D\xfb\xbbJ.\xc8\r1\xf1X\xb4\xaa\xf0\xe6\x07\x06\x88/\xcbb\xdf\x8cS\xa7hnH\xb0\x0c\xad\xbc\x8f\xca\xe5N\x9f\xfb\xa0|\xe8\xb9D\xd7\x84\xc7\xd1\x18\x93\xe5b\xc7\xe4\x8dI\x9aO\x9c\x92Q\'Ch\xf3%\xccRc\xcc\xfa\xe6\xbab9\x1f\xeaR9\x0b\xd3|Q\xc0\x1dpVC\x8c\xce\x03l\xf0\x85v\xd3\xa1r7\x92\x853\x03\x86\x02CF\xcb\xd4X\xd5\x99.,\xe6\xd4)\xa2\x9c\xe2h(\xa2\x8a\x18V\xf6\x9f\x9aJx\x95\xcdT\xe4jl5\x95%~\\\xe8\xc3\'^\\\xd4Q}\xc8\xcb\x12\xcb*7H\xb3i\x94J\xb7S\x01\x1b\x08B\xae\xbe5\xd1\x94/\xba"\xbfy\x10\x8f\x02\xc1N\xdd\xe4A5\xb9A\xa9\xa8\x98\xbdF\x9bf\xfb\x84pe\xbd:L\x88\x7f\xae\xee\xe9\x9d\xfa\x03b%\xb3\xda\xdd\x94\xc1\x95B5\x14\x91@\x93\xbeZ\xe6\xe6\x1au=\xf2(?^\xba*\xab\

In [8]:
print(collections.Counter(keystream).most_common(8))
# the tuples (a, b) represents the ascii code (a) and the relative frequency (b) inside the bytearray 
# uniform distribution => pseudorandom stuff

[(43, 122), (201, 119), (36, 109), (171, 108), (213, 107), (3, 107), (205, 107), (75, 107)]


In [9]:
AES_BLOCK = 16 # 128 bit blocks
max_length = len(keystream)//10 # limiting the max length for performance purpose

# trying to exploit the counter iterating through AES blocks
index = 0
for block in [keystream [i : len(keystream)] for i in range(0, max_length, AES_BLOCK)]:
    index += 1
    temp = np.bitwise_xor(block,challenge[0:len(block)])
    if 32 in collections.Counter(temp).most_common(8)[0]: # checking for ASCII Code 32 => it is the Space char
        print("Initial Value shift: " + str(index - 1))
        print(temp[:200].tostring())

Initial Value shift: 29
b"1\x0eL-\xdb]9C\x18I\x99\xb1w\xea\xff\x884*\xa7<U`\x08\xe2\xf3P`\xa4\xe6]A\xdaD]\xb9I\x93\x10O#o?\xaf\x81\x97\xaf\xfc\xaf\xee\xf2\x94\xfd\xd6\xa8~\xdcp\x04t\x80\xfa\x08\x00Q\x9a\x01\x95\x1e\x86\xf4r!v1\xd2Lpc\xea?'\xf6G\xa1\x84\xa1iRH\xbf\x8c\x86\x05\x9e\xdey\xd9Vx\x08\xab\xe7\xb6\xbc\x98l\xdc#l \x80\x1b\xb3\x15\x8f\x95\x83\xc5\xc2\xe3\xd7p\xe2\xbfe'\x9e\xc0\x00\x16*\xe0\x17+\xffnr\xc3\xaeD_\xce(\xec\x86q\x1a\\\xbe\xae\xa8\x10\xcc\xd0m\n\xd2\xca\xd0\x8f#\xf9\xb6)\x11~\xbb[\xbdI\xdbm\n\xbc\xb7Y!=B\x0bn\xffx\xba]}r\x94\x17n\xe5\x06\xac\x14\xf5\x8d-h\xb8\x8e"
Initial Value shift: 30
b'\xacA\x13\xa2\xec\xf8Y\x8f\xfa\xc5\xe9:\xec\x0e\xf3\x16~\x99\xb8\x9cUR\x02\xaa7\x9d\xf3\xd8\x84\xaa=7\x89d\xb3\x05)\xe2\xba3@\xd8\xea\xbb\n -\'\xa4\'\xc3\x14\xab\xd76\x14]\xb4\x17\x8c\xa6\x08"\xc0\xa5\xfb4Ui\x97\x8e\xf8\xd3\xf7\xe1\xeeY(\xd8\x029\xa0\xfem\x99\x1aO\xd2\xd5S\xde\xd5\xc5\xe3j5\x12O\x8e.-\xa1\x0e\xe7\x1a\xf8\xcd\x05N\x1e\xa1E\xa8~\xd2DW\x8bXC?\xceO\xce\xf020\'\xd8\xc4\x9

In [10]:
shift = AES_BLOCK * 80
t = np.bitwise_xor(keystream[ shift : len(keystream)],challenge[0:len(keystream) - shift])
print(t[:1000].tostring()) #limiting the output and decoding in readable format

b'THE RIME OF THE ANCIENT MARINER\r\n\r\nIN SEVEN PARTS\r\n\r\nBy Samuel Taylor Coleridge\r\n\r\n\r\n\r\nPART THE FIRST.\r\n\r\n     It is an ancient Mariner,\r\n     And he stoppeth one of three.\r\n     "By thy long grey beard and glittering eye,\r\n     Now wherefore stopp\'st thou me?\r\n\r\n     "The Bridegroom\'s doors are opened wide,\r\n     And I am next of kin;\r\n     The guests are met, the feast is set:\r\n     May\'st hear the merry din."\r\n\r\n     He holds him with his skinny hand,\r\n     "There was a ship," quoth he.\r\n     "Hold off! unhand me, grey-beard loon!"\r\n     Eftsoons his hand dropt he.\r\n\r\n     He holds him with his glittering eye--\r\n     The Wedding-Guest stood still,\r\n     And listens like a three years child:\r\n     The Mariner hath his will.\r\n\r\n     The Wedding-Guest sat on a stone:\r\n     He cannot chuse but hear;\r\n     And thus spake on that ancient man,\r\n     The bright-eyed Mariner.\r\n\r\n     The ship was cheered, the harbour 

# Mission Complete